In [ ]:
# default_exp load_gti
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# GTI, or Good Time Interval

> Process GTI information

In [ ]:
#export
from astropy.io import fits
import numpy as np
import pandas as pd

from light_curves.config import MJD

def get_gti(config, gti_files):
    """Combine the GTI intervals that fall within the gti range
    Return a function that tests a list of times
    """
    gti_files = list(gti_files.glob('*.fits'))
    if config.verbose>1:
        print(f'Processing {len(gti_files)} FITS files with GTI information ... ', end='')
    starts=[]
    stops=[]
    for i, ft1 in enumerate(gti_files):
        with fits.open(ft1) as hdu:
            gti_data = hdu['GTI'].data
            start = gti_data.START
            if i>0:
                assert start[0]>= stops[-1][-1], f'file {ft1} has start time not following preceding file'
            starts.append(start)
            stops.append( gti_data.STOP)
    start = MJD(np.concatenate(starts))
    stop  = MJD(np.concatenate(stops))

    if config.verbose>1:
        livetime = (stop-start).sum()
        print( f' {len(gti_files)} files, {len(start)} intervals with'\
               f' {int(livetime):,} days live time')

    sel = slice(None)
    if config.mjd_range is not None:
        a,b =  np.searchsorted(start, config.mjd_range)
        if a>0 or b<len(start):
            if config.verbose>1:
                print(f'\tcut from {len(start):,} to {a} - {b}, or {b-a:,} entries after MJD range selection')
            sel = slice(a,b)


    class GTI(object):
        """ functor class that tests for being in the GTI range
        """
        def __init__(config, start, stop):
            # prepare single merged array with even, odd entries start and stop
            a,b =start, stop
            config.fraction = np.sum(b-a)/(b[-1]-a[0])
            assert len(a)==len(b)
            config.g = np.array([a,b]).T.flatten()
            assert np.sum(np.diff(config.g)<0)==0, 'Bad GTI ordering'

        def __call__(config, time):
            # use digitize to find if in good/bad interval by odd/even
            x = np.digitize(time, config.g)
            return np.bitwise_and(x,1).astype(bool)

        def __repr__(config):
            return  f'{config.__class__.__name__} MJD range: {config.g[0]:.2f}-{config.g[-1]:.2f}'\
                    f', good fraction {config.fraction:.2f} '

    gti =  GTI(start[sel],stop[sel])
    if config.verbose>1:
        print(f'\t{gti}')
    return gti

In [ ]:
from light_curves.config import Config, Files
config = Config(mjd_range=(54000,55000))
gti_files = Files().gti
if gti_files.exists():
    print(f'Test with GTI files from {gti_files}')
    gti_list = get_gti(config, gti_files)
else:
    print('No files, so no test')

Test with GTI files from /home/burnett/work/lat-data/binned
Processing 11 FITS files with GTI information ...  11 files, 63635 intervals with 3,322 days live time
	cut from 63,635 to 0 - 4816, or 4,816 entries after MJD range selection
	GTI MJD range: 54682.66-55000.03, good fraction 0.82 


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_load_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 07_cells.ipynb.
Converted 09_poisson.ipynb.
Converted 10_loglike.ipynb.
Converted 11_lightcurve.ipynb.
Converted index.ipynb.
Mon Dec  7 08:17:50 PST 2020
